# Q-2. 
```
Imagine you have a dataset where you have different features like Age ,
Gender , Height , Weight , BMI , and Blood Pressure and you have to classify the people into
different classes like Normal , Overweight , Obesity , Underweight , and Extreme Obesity by using
any 4 different classification algorithms. Now you have to build a model which
can classify people into different classes.
Dataset This is the Dataset You can use this dataset for this question.
```

In [31]:
import pandas as pd

In [32]:
df = pd.read_csv("ObesityDataSet_raw_and_data_sinthetic.csv")

In [33]:
X = df.drop(["NObeyesdad"], axis=1)
y = df["NObeyesdad"]

In [34]:
y

0             Normal_Weight
1             Normal_Weight
2             Normal_Weight
3        Overweight_Level_I
4       Overweight_Level_II
               ...         
2106       Obesity_Type_III
2107       Obesity_Type_III
2108       Obesity_Type_III
2109       Obesity_Type_III
2110       Obesity_Type_III
Name: NObeyesdad, Length: 2111, dtype: object

In [35]:
cat_cols = list(df.select_dtypes(include="object"))[:-1]

In [37]:
cat_cols

['Gender',
 'family_history_with_overweight',
 'FAVC',
 'CAEC',
 'SMOKE',
 'SCC',
 'CALC',
 'MTRANS']

In [38]:
num_cols = list(df.select_dtypes(exclude="object"))

In [39]:
num_cols

['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, LabelEncoder

target_col = ['NObeyesdad']

cat_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OrdinalEncoder())
])

num_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
])

target_pipeline = Pipeline(steps=[
    ("encode", LabelEncoder())
])

preprocessor = ColumnTransformer([
    ("cat", cat_pipeline, cat_cols),
    ("num", num_pipeline, num_cols)
])

target_preprocessor = ColumnTransformer([
    ("target", target_pipeline, target_col)
])

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [25]:
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [26]:
encoder = LabelEncoder()
y_train_processed = encoder.fit_transform(y_train)
y_test_processed = encoder.transform(y_test)

In [27]:
# Basic Import
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
import xgboost as xgb
from lightgbm import LGBMClassifier

In [28]:
models = {
    'etc': ExtraTreesClassifier(random_state=42, n_estimators=201, criterion="gini"),
    'rfc': RandomForestClassifier(random_state=42, n_estimators=101),
    'gbc': GradientBoostingClassifier(random_state=42, n_estimators=201),
    'cat': CatBoostClassifier(verbose=0),
    "lgbm": LGBMClassifier(random_state=42),
}

macro_average_list = []
trained_models_list = []

# looping through dictionary, create model and evaluates it
for model in list(models.values()):
    model.fit(X_train_processed, y_train_processed)
    y_pred = model.predict(X_test_processed)
    report = classification_report(y_test_processed, y_pred, output_dict=True)
    print(f"{model}:\n{classification_report(y_test_processed, y_pred)}")
    macro_average = report['macro avg']['f1-score']
    macro_average_list.append(macro_average)
    trained_models_list.append(model)

ExtraTreesClassifier(n_estimators=201, random_state=42):
              precision    recall  f1-score   support

           0       0.93      0.97      0.95        86
           1       0.81      0.87      0.84        93
           2       0.99      0.98      0.99       102
           3       0.99      0.99      0.99        88
           4       1.00      0.99      0.99        98
           5       0.90      0.81      0.85        88
           6       0.90      0.91      0.91        79

    accuracy                           0.93       634
   macro avg       0.93      0.93      0.93       634
weighted avg       0.93      0.93      0.93       634

RandomForestClassifier(n_estimators=101, random_state=42):
              precision    recall  f1-score   support

           0       0.99      0.95      0.97        86
           1       0.81      0.92      0.86        93
           2       0.98      0.94      0.96       102
           3       0.97      0.99      0.98        88
           4    

In [29]:
macro_avg_max_value = max(macro_average_list)
macro_avg_max_index = macro_average_list.index(macro_avg_max_value)
best_model = trained_models_list[macro_avg_max_index]
best_model_name = list(models.keys())[macro_avg_max_index]

In [30]:
print(f"{best_model_name}: {round(macro_avg_max_value, 2)*100}")

lgbm: 96.0


In [49]:
df["NObeyesdad"].value_counts().index

Index(['Obesity_Type_I', 'Obesity_Type_III', 'Obesity_Type_II',
       'Overweight_Level_I', 'Overweight_Level_II', 'Normal_Weight',
       'Insufficient_Weight'],
      dtype='object')